## Python Google Big Query Example

This notebook explores your ability to connect a Python notebook to a Google Big Query instance. *NOTE*: Before you try to run this notebook, you need to _complete_ steps 1 and 2 from this [page](https://cloud.google.com/bigquery/docs/reference/libraries#client-libraries-install-python). For step 2, "Setting up Authentication", I recommend following the console directions. Pay attention to where you save your JSON authentication file, since you'll need to tell Python how to find it. 

There's a second part of step 2, where you set an environment variable at the command line to your credentials. I was never able to make that work on my machine, so I'm using a direct path in my code below.That second step of that requires a bit of command line work. 

In [1]:
# Do our imports for the code
from google.cloud import bigquery
from google.oauth2 import service_account

In [11]:
# These first two values will be different on your machine. 
service_path = "/Users/chandler/Dropbox/Teaching/"
service_file = 'umt-msba-037daf11ee16.json' # change this to your authentication information  
gbq_proj_id = 'umt-msba' # change this to your poroject. 

# And this should stay the same. 
private_key =service_path + service_file

In [13]:
# Now we pass in our credentials so that Python has permission to access our project.
credentials = service_account.Credentials.from_service_account_file(service_path + service_file)

In [14]:
# And finally we establish our connection
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

At this point, our `client` variable holds a connection to the project. This is client is similar to a file handle--it allows you to "talk" to the project. Let's begin by writing a query and running it. 

---

### Querying Data

In this next section we'll query some data from our GBQ instance. In order to have something to play with, we'll get a very approximate measure of spend by `card_no` for a portion of the July 2013 data. 

In [15]:
# An example query. This uses a trick that Python concatenates adjacent string literals for you.
# Note, you'll need to update the project_id and dataset_id in the query below (and throughout)
query = (
    "SELECT card_no, sum(total) as TotalSales "
    "FROM `umt-msba.wedge_example.transactions_201307_small` "
    "WHERE total > 0 AND trans_type = 'I' "
    "GROUP BY card_no "
    "ORDER BY TotalSales DESC "
)

# And we execute queries with `client.query`
query_job = client.query(
    query,
    location="US",
)

Once we've called `client.query` and assigned the results, we can iterate over the result set. In this case we'll print the first $n$ rows in a nice format. Note that `row`, the name we give to the items produced when we iterate over  `query_job` is a tuple of length two. 

In [16]:
cards = []

for idx, row in enumerate(query_job) :
    card, sales = row
    print("Card {:.0f} spent {:,.2f} dollars.".format(card,sales))
    cards.append(card)
    if idx == 25 :
        break

Card 3 spent 144,021.33 dollars.
Card 12539 spent 15,281.17 dollars.
Card 10499 spent 7,924.94 dollars.
Card 14140 spent 7,152.03 dollars.
Card 25016 spent 4,502.70 dollars.
Card 20074 spent 4,264.39 dollars.
Card 14987 spent 3,169.13 dollars.
Card 19501 spent 1,941.07 dollars.
Card 10504 spent 1,052.63 dollars.
Card 13758 spent 1,034.45 dollars.
Card 50056 spent 698.63 dollars.
Card 21517 spent 691.69 dollars.
Card 18389 spent 644.69 dollars.
Card 14345 spent 631.40 dollars.
Card 18876 spent 611.77 dollars.
Card 18489 spent 610.16 dollars.
Card 19750 spent 600.29 dollars.
Card 15876 spent 588.82 dollars.
Card 25334 spent 582.00 dollars.
Card 15967 spent 570.89 dollars.
Card 13341 spent 558.53 dollars.
Card 25181 spent 547.85 dollars.
Card 20192 spent 547.45 dollars.
Card 10714 spent 538.93 dollars.
Card 20197 spent 503.11 dollars.
Card 50021 spent 501.24 dollars.


---

### Uploading Data

To add data to the dataset, we'll need to create a table and put data in it. We'll do a simple example here to illustrate the concept. 

#### Uploading Manually

This is the easist way, but it only works for small files. Go to the console, and click on your project (the one that you replaced `umt-msba` with in the above) and then on your dataset (like `wedge_example`). You'll see a blue "+" sign (in the new UI). Click on that and it will open a "create table" prompt. 

Once that's open, here are the options to choose as you go down: 

1. Create from "Upload"
1. Select the file `department_lookup.csv` that's in the same folder as this notebook.
1. Give your table a unique name. I'm using `chandler_test1` here.  
1. Select "Auto detect" under "Schema"

You don't need to worry about the other fields and checkboxes. 

In [ ]:
# You should have data in the file that you can see via the console. Let's test it here.
query = (
    "SELECT * "
    "FROM `umt-msba.wedge_example.chandler_test1` " # change to *your*` project ID, data set ID, and table name! 
)

results = client.query(query)

print("Let's print it ugly.")
print(list(results))

print("\nAnd now let's print it pretty.\n")

for dept_id, dept_name in results :
    print(f"{dept_name} has id {dept_id}.")


### Manual Table Creation, Programmatic Upload

Now we'll do something similar. We'll create the table manually but upload the data programmatically.  

So go through similar steps to create the table via the console. Open up the "create table" prompt. 

Once that's open, here are the options to choose as you go down: 

1. Create from "Empty Table"
1. Give your table a unique name. I'm using `chandler_test2`. 
1. Add two fields. One called "department" that will be an integer and one called "dept_name" that will be a string.

You don't need to worry about the other fields and checkboxes. 

In [ ]:
# Let's start by querying the data and seeing that it's empty.

query = (
    "SELECT * "
    "FROM `umt-msba.wedge_example.chandler_test2` " # change to *your*` project ID, data set ID, and table name!
)

In [ ]:
# and now run the query
results = client.query(query)

In [ ]:
print(list(results)) # Should be an empty list

Now let's put some data in the table. This is a bit messy *and* only works for small data sets, so I'll show you how this is done, but we won't probably use this technique in the future. This code is taken directly from the GBQ [docs on uploading data](https://cloud.google.com/bigquery/docs/loading-data-local#limitations).

In [ ]:
dataset_ref = client.dataset("wedge_example") # set up references to the dataset and table.
table_ref = dataset_ref.table("chandler_test2") # use your name

Now we need to establish a job that will run this import for us. A clunky step.

In [ ]:
job_config = bigquery.LoadJobConfig() 
job_config.source_format = bigquery.SourceFormat.CSV
job_config.skip_leading_rows = 1
job_config.autodetect = True
input_file = "department_lookup.csv"

In [ ]:
with open(input_file, "rb") as source_file:
    job = client.load_table_from_file(source_file, table_ref, job_config=job_config)
    
job.result()  # Waits for table load to complete.

print("Loaded {} rows into {}:{}.".format(job.output_rows, dataset_ref, table_ref))

Now let's query this table and see if it worked.

In [ ]:
# You should have data in the file that you can see via the console. Let's test it here.
query = (
    "SELECT * "
    "FROM `umt-msba.wedge_example.chandler_test2` " # change to *your*` project ID, data set ID, and table name! 
)

results = client.query(query)
print(list(results))

---

When you're working on the Wedge project, the table schema is *much* more complicated. If you've reached this point in the notebook and have some extra time, see if you can download the table schema from one of our Wedge tables.  